In [1]:
import os
import pandas as pd
import janitor
import json
import datetime
from tqdm.notebook import tqdm
from utilities import read_json

import warnings

warnings.filterwarnings("ignore")

output_path = "../output/cache_pypi_downloads_payload_allinstallers/"

In [2]:
df_payloads = pd.DataFrame()
for filename in tqdm(os.listdir(output_path)):
    if filename.endswith(".json"):
        filename = filename.replace(".json", "")
        _, date, _ = filename.split("_", maxsplit=3)
        
        filepath = os.path.join(output_path, filename)
        try:
            data = read_json(filepath)
        except json.JSONDecodeError as e:
            # Handle the case where JSON is not valid
            print(f"Failed to decode JSON: {e} for {filename}")
            continue
            
        package = data["args"][-2]
        downloads = data["rows"]
        if len(downloads)!=0:
            _df = (
                pd.DataFrame(downloads)
                # Transpose so downloads are columns
                .set_index('installer_name').T
                .rename_axis(None, axis=1)
                .assign(pkg=package)
                .assign(date=date)
            )

        else:  # no downloads on that day
            _df = pd.DataFrame([{"pkg": package, "date": date}])
        
        df_payloads = pd.concat([df_payloads, _df], ignore_index=True)

df_payloads.head(3)

  0%|          | 0/118180 [00:00<?, ?it/s]

Failed to decode JSON: Expecting value: line 1 column 1 (char 0) for 2023-09-12_2023-09-13_hoyo-daily-logins-helper
Failed to decode JSON: Expecting value: line 1 column 1 (char 0) for 2023-09-25_2023-09-26_cset


,bandersnatch,Browser,pkg,date,pip,None,requests,Nexus,conda,devpi,pdm,Artifactory,setuptools
0,5.0,1.0,cake-utils-pkg-poorva278,2023-05-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,134.0,54.0,sharetop,2023-08-27,80.0,52.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN
2,28.0,1.0,readable-number,2023-05-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
human_installers = [
    "pip",
    "setuptools",
    "Nexus",
    "pdm",
    "Homebrew",
    "Artifactory",
    "OS",
    "Bazel",
    "pex",
    "conda",
    "chaquopy"
]

df_dl = (
    df_payloads
    .reorder_columns(["pkg", "date"])
    .sort_values(["pkg", "date"], ignore_index=True)
    .fillna(0)
    # =================================================================
    # Filter down to organic downloads
    .set_index(["pkg", "date"])
    .filter(human_installers)
    # Sum all human downloads
    .assign(download_count=lambda x: x.sum(axis=1))
    .reset_index()
    .select_columns(["pkg", "date", "download_count"])
)
df_dl

,pkg,date,download_count
0,a-pandas-ex-df2htmlstring,2023-04-25,0.0
1,a-pandas-ex-df2htmlstring,2023-04-26,0.0
2,a-pandas-ex-df2htmlstring,2023-04-27,0.0
3,a-pandas-ex-df2htmlstring,2023-04-28,7.0
4,a-pandas-ex-df2htmlstring,2023-04-29,7.0
...,...,...,...
118173,zoomaker,2023-10-27,0.0
118174,zoomaker,2023-10-28,0.0
118175,zoomaker,2023-10-29,0.0
118176,zoomaker,2023-10-30,0.0


In [4]:
# Read in 100 random sample
df_random_sample = (
    pd.read_csv("../input/random_sample.csv")
    .assign(slug=lambda df: df["github"].str.replace("https://github.com/", ""))
    .assign(
        slug=lambda df: df["slug"].apply(lambda x: x[:-1] if x.endswith("/") else x)
    )
    .assign(slug=lambda df: df["slug"].str.lower().str.strip())
    .assign(fileslug=lambda df: df["slug"].str.replace("/", "_"))
    .assign(treated=1)
    # Assign the two types
    .assign(boughtstars=lambda df: (df.index < 25).astype(int))
    .assign(treated2=lambda df: df["boughtstars"] + df["treated"])
)
df_random_sample.head(3)

,pkg,return_code,github_url,homepage,earliest_release,gh_url_check,github,pypi,label,slug,fileslug,treated,boughtstars,treated2
0,bird-ospf-link-db-parser,200.0,https://github.com/Andrew-Dickinson/bird-ospf-...,NaN,2023-04-29T07:23:44,1.0,https://github.com/Andrew-Dickinson/bird-ospf-...,https://pypi.org/project/bird-ospf-link-db-par...,bird-ospf-link-db-parser\r\nhttps://github.com...,andrew-dickinson/bird-ospf-link-db-parser,andrew-dickinson_bird-ospf-link-db-parser,1,1,2
1,asciicli,200.0,https://github.com/mrq-andras/asciicli,https://github.com/mrq-andras/asciicli,2023-04-28T07:22:55,1.0,https://github.com/mrq-andras/asciicli,https://pypi.org/project/asciicli/#history,asciicli\r\nhttps://github.com/mrq-andras/asci...,mrq-andras/asciicli,mrq-andras_asciicli,1,1,2
2,bdpotentiometer,200.0,https://github.com/bond-anton/BDPotentiometer,https://github.com/bond-anton/BDPotentiometer,2023-04-27T06:35:18,1.0,https://github.com/bond-anton/BDPotentiometer,https://pypi.org/project/bdpotentiometer/#history,bdpotentiometer\r\nhttps://github.com/bond-ant...,bond-anton/bdpotentiometer,bond-anton_bdpotentiometer,1,1,2


In [5]:
df = (
    df_dl
    # =================================================================
    # Fix dates
    .assign(date=lambda df: pd.to_datetime(df["date"]))
#     .query("date <= @datetime.date(2023, 10, 31)")
    .sort_values(["pkg", "date"], ignore_index=True)
    # =================================================================
    # Fill-in missing package-dates w/ zeroes
    .set_index(["pkg", "date"])
    .pipe(
        lambda df_: df_.reindex(
            pd.MultiIndex.from_product(
                [
                    df_.index.get_level_values("pkg").unique(),
                    pd.date_range("2023-04-25", "2023-10-31"),
                ],
                names=["pkg", "date"],
            )
        )
    )
    .fillna({"download_count": 0})
    .reset_index()
    # =================================================================
    # Get cumulative downloads
    .assign(tt_downloads=lambda df: df.groupby("pkg")["download_count"].cumsum())
    # =================================================================
    # Merge to get treatment status
    .merge(
        df_random_sample.select_columns("pkg", "treated", "boughtstars", "treated2"),
        how="left",
        on="pkg",
        validate="m:1",
    )
    .assign(treated=lambda df: df["treated"].fillna(0).apply(int))
    .assign(boughtstars=lambda df: df["boughtstars"].fillna(0).apply(int))
    .assign(treated2=lambda df: df["treated2"].fillna(0).apply(int))
    .assign(date=lambda df: pd.to_datetime(df["date"]))
    .assign(date=lambda df: [dtobj.date() for dtobj in df["date"]])
)
# Assert that 100 repos are assigned as treated
assert 100 == len(df.drop_duplicates("pkg").query("treated==1"))
assert 25 == len(df.drop_duplicates("pkg").query("treated2==2"))
# Assert complete panel
assert len(df) == (df["pkg"].nunique() * df["date"].nunique())
df

,pkg,date,download_count,tt_downloads,treated,boughtstars,treated2
0,a-pandas-ex-df2htmlstring,2023-04-25,0.0,0.0,0,0,0
1,a-pandas-ex-df2htmlstring,2023-04-26,0.0,0.0,0,0,0
2,a-pandas-ex-df2htmlstring,2023-04-27,0.0,0.0,0,0,0
3,a-pandas-ex-df2htmlstring,2023-04-28,7.0,7.0,0,0,0
4,a-pandas-ex-df2htmlstring,2023-04-29,7.0,14.0,0,0,0
...,...,...,...,...,...,...,...
118175,zoomaker,2023-10-27,0.0,338.0,0,0,0
118176,zoomaker,2023-10-28,0.0,338.0,0,0,0
118177,zoomaker,2023-10-29,0.0,338.0,0,0,0
118178,zoomaker,2023-10-30,0.0,338.0,0,0,0


In [6]:
df.to_csv("../output/pkg_human_downloads.csv", index=False)